# ノード数変更のスケジュールを設定する

---

計算ノード数を増減させるスケジュール設定を行います。

## 概要

計算ノードの起動数を変更するスケジュールの設定を行います。利用頻度の低い時間帯に計算ノードの起動数を減らすことで、クラウドプロバイダの課金や使用電力量を抑えるようにします。

このnotebookで設定を行う対象を次図に示します。

![構成図](images/ohpc-811-00.png)

計算ノード数を制御するコンテナをmasterノードで実行し、VCノードの起動数の変更やSlurmのノード状態の更新を行います。制御コンテナはsystemd timerから一定時間（１５分）毎に起動されます。起動されたコンテナでは現在の計算ノードの状態を確認し、変更が必要な場合のみVCノードの起動、停止やSlurmのノード状態の更新を行います。

### 計算ノード数

計算ノードの利用頻度は日中と夜間、平日と週末など時間帯によって異なることが考えられます。このnotebookでは特定の時間帯の計算ノード数を通常時から変更する設定を行います。

通常時のノード数は「[3.1 通常時のノード数](#通常時のノード数)」で指定します。計算ノード数を変更する時間帯とそのノード数は「[3.2 スケジュールの指定](#スケジュールの指定)」で指定します。時間帯を指定する粒度は日毎、週毎、月毎、年毎、特定日時を選択できます。またノード数の変更スケジュールは複数の設定を指定することができます。

このnotebookでスケジュール設定を行なった後に臨時のスケジュールなどを追加する場合は「812-ノード数のスケジュール設定を変更する.ipynb」で設定を変更してください。

### VCノードの停止方法

計算ノードはVCノードとして管理されています。そのためVCPSDKを利用することでノードの起動、停止などの操作を実現できます。VCノードの停止方法には、ノードを削除する方法とノードの電源オフにする方法があります。どちらの方法を用いてノードの停止を行うかは「[3.3 ノードの停止方法](#ノードの停止方法)」で指定することができます。

> VCノードの電源操作は一部のクラウドプロバイダのみ利用可能な機能です。電源操作をサポートしているかについては[OCSユーザーズマニュアル](https://nii-gakunin-cloud.github.io/ocs-docs/usermanual/appendix.html#vcp%E3%81%AB%E3%81%8A%E3%81%91%E3%82%8B%E3%82%AF%E3%83%A9%E3%82%A6%E3%83%89%E3%83%97%E3%83%AD%E3%83%90%E3%82%A4%E3%83%80%E3%81%A8%E3%82%AA%E3%83%97%E3%82%B7%E3%83%A7%E3%83%8A%E3%83%AB%E6%A9%9F%E8%83%BD%E3%81%AE%E5%AF%BE%E5%BF%9C)を確認してください。

### ノード停止の準備期間

Slurmのノード状態を[DRAIN](https://slurm.schedmd.com/scontrol.html#OPT_DRAIN)にすることで、そのノードで新たなジョブが開始されないようにすることができます（既存のジョブは完了するまで実行される）。ノードを停止する前の一定期間、ノード状態をDRAINにするように設定することができます。
停止前にDRAINにする時間を「[3.4 ノード停止前にDRAIN状態とする時間](#ノード停止前にDRAIN状態とする時間)」で指定してください。

## パラメータの指定

計算ノードのスケジュールを登録するのに必要となるパラメータを指定します。

### UnitGroup名

対象となるOpenHPCのUnitGroup名を指定します。

VCノードを作成時に指定したUnitGroup名を確認するために `group_vars` ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

UnitGroup名を指定してください。

In [ ]:
# (例)
# ugroup_name = 'OpenHPC'

ugroup_name = 

#### チェック

指定されたグループ名が妥当なものであることをチェックします。

OpenHPCのmasterノードにアクセスできることを確認します。

In [ ]:
target_group = f'{ugroup_name}_master'
!ansible {target_group} -m ping

masterノードにて管理者として操作できることを確認します。

In [ ]:
!ansible {target_group} -b -a 'whoami'

指定されたUnitGroupで、既にスケジュール設定が行われていないことを確認します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)
if gvars.get("vcnode_schedule") is not None:
    raise RuntimeError("既にスケジュール設定が行われています")

> 設定済みのスケジュールを変更するにはこのnotebookではなく「812-計算ノード数のスケジュールを変更する.ipynb」を実行してください。

### VCCアクセストークンの入力

VCノードを操作するためにVC Controller(VCC)のアクセストークンが必要となります。
次のセルを実行すると表示される入力枠にVCCのアクセストークンを入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
ERROR - config vc failed: http_status(403)
ERROR - 2025/04/01 00:00:00 UTC: VCPAuthException: xxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

## スケジュール定義

### 通常時のノード数

通常時の計算ノード数を指定します。OpenHPCの構築時に指定した最大ノード数を超える値を指定することはできません。また、現在の計算ノード数を通常時のノード数とする場合はこの節をスキップして次の節に進むことが
できます。

通常時の計算ノード数を指定する前に、計算ノード数の最大値を確認します。次のセルを実行すると最大ノード数が表示されます。

In [ ]:
print(gvars["max_compute_nodes"])

現在の計算ノード数を確認します。

In [ ]:
print(gvars["compute_nodes"])

通常時の計算ノード数を次のセルで指定してください。

In [ ]:
# (例)
# schedule_default_compute_nodes = 10

schedule_default_compute_nodes = 

### スケジュールの指定

計算ノード数を変更する期間とそのノード数を指定します。

ひとつのスケジュールを定義するには、期間を`begin`(開始時刻)と`end`(終了時刻)で指定し、その期間における計算ノード数を`node_count`で指定します。この３つの値をpythonのdictで指定します。例えば毎日22:00から翌朝7:00までは計算ノード数を0にするスケジュールは以下のような値で定義します。

```python
{
    "begin": "22:00",
    "end": "7:00",
    "node_count": 0,
}
```

期間は日毎、週毎、月毎、年毎、特定日時の指定が行えます。

日毎の場合は`22:00`のように`{時}:{分}`の形式で指定します。

週毎の場合は`Sat 0:00`のように`{曜日} {時}:{分}`の形式で指定します。曜日は`Sunday`、`日曜日`、`Sun`、`日`のような値で指定を行います。

月毎の場合は`1日 0:00`のように`{日} {時}:{分}`の形式で指定します。日は`1日`、`1`のような値で指定を行います。

年毎の場合は`12/29 0:00`のように`{月}/{日} {時}:{分}`の形式で指定します。

特定日時の場合は`2025/4/1 0:00`のように`{年}/{月}/{日} {時}:{分}`の形式で指定します。

期間の開始と終了は同じ粒度（日毎、週毎、月毎、年毎、特定日時）を指定する必要があります。

次のセルでスケジュール定義のリストを指定してください。リストの個々の要素は上で説明した`begin`, `end`, `node_count`をキーに持つdictになります。複数のスケジュール期間が重複した場合、リストで先に指定したものが優先されます。

In [ ]:
schedule_list = [
# (例)
#    {"begin": "22:00", "end": "7:00", "node_count": 0},                    # 日毎のスケジュール 22:00--7:00
#    {"begin": "Sat 0:00", "end": "Mon 0:00", "node_count": 0},             # 週毎のスケジュール 土曜日0:00--月曜日0:00
#    {"begin": "1日 0:00", "end": "1日 10:00", "node_count": 0},            # 月毎のスケジュール 1日0:00--1日10:00
#    {"begin": "12/29 0:00", "end": "1/4 0:00", "node_count": 0},           # 年毎のスケジュール 12月29日0:00--1月4日0:00
#    {"begin": "2025/4/1 0:00", "end": "2025/4/2 0:00", "node_count": 0},   # 特定日時のスケジュール 2025/4月1日0:00--2025/4月2日0:00

]

設定されたスケジュールをチェックするための準備を行います。

In [ ]:
from pathlib import Path

!ansible-playbook -v -e venv_dir={str(Path.cwd() / ".venv")} playbooks/setup-local-venv.yml

リスト`schedule_list`に指定したスケジュールをチェックします。次のセルを実行してエラーにならないことを確認してください。指定されたスケジュールのリストに問題ない場合は設定した内容を表示します。意図したスケジュールが指定できていることを確認してください。

In [ ]:
import sys
%run scripts/schedule.py

params = {name: value for name, value in vars().items() if name.startswith("schedule")}
params["schedule_down_type"] = schedule_down_type if "schedule_down_type" in vars() else "power_down"
try:
    description_schedule(ugroup_name, params, "node_count")
except ValueError as e:
    print(f"ERROR: {e}", file=sys.stderr)
    raise

### ノードの停止方法

ノードの停止方法は、ノードを削除する方法と、ノードの電源をオフにする方法のどちらかを選択できます。

次のセルで`deleted`(削除)と`power_down`(電源オフ)のどちらかを指定してください。

In [ ]:
# (例)
# schedule_down_type = "deleted"           # ノードを削除する
# schedule_down_type = "power_down"  # ノードの電源をオフする

schedule_down_type = 

### ノード停止前にDRAIN状態とする時間

ノードを停止する前に、新たなジョブを受け付けを停止する期間を設けます。これは、ノードを停止する前の一定時間Slurmのノード状態を`DRAIN`にすることで実現します。

ノード状態を`DRAIN`にする時間を次のセルで指定してください。指定の単位は分となります。

In [ ]:
# (例)
# schedule_drain_time = 60

schedule_drain_time = 

### 設定内容の確認

ここまで設定した内容を確認します。

In [ ]:
%run scripts/schedule.py
params = {name: value for name, value in vars().items() if name.startswith("schedule")}
description_schedule(ugroup_name, params)

### 保存

定義したスケジュールをgroup_varsに保存します。

In [ ]:
%run scripts/schedule.py
params = {name: value for name, value in vars().items() if name.startswith("schedule")}
update_group_vars(
    ugroup_name,
    vcnode_schedule=get_schedule_definition(ugroup_name, params),
)

## 配備

スケジュール設定に関する資材をmasterノードに配備してサービスを開始するように設定します。

次のセルを実行すると資材の配備とサービスの開始を行います。

In [ ]:
import json
from pathlib import Path
from tempfile import TemporaryDirectory

with TemporaryDirectory() as work_dir:
    vars_path = Path(work_dir) / "params.json"
    with vars_path.open(mode="w") as f:
        json.dump({"vcc_access_token": vcc_access_token}, f)
    !ansible-playbook -Dv -l {target_group} -e @{str(vars_path)} playbooks/setup-schedule.yml

OpenHPCのmasterノードに配備した資材を確認します。

In [ ]:
!ansible {target_group} -b -a 'tree chdir=/srv/compute-schedule'

開始したsystemdタイマーの状態を確認します。通常は１５分ごとに計算ノード状態の確認とノード数の更新を行います。

In [ ]:
!ansible {target_group} -b -a 'systemctl list-timers ohpc-compute-schedule'